In [1]:
from temporal_grounding.temporal_grounding import *

In [2]:
video_path = r"C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\trimmed-videos\broadcast_2_period_1.mp4"
save_path = r"data/example_timestamps.json"

extract_timestamps_from_video(video_path=video_path, save_path=save_path)

Finding time-remaining ROI for video at C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\trimmed-videos\broadcast_2_period_1.mp4


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:06<?, ?it/s]


Extracting timestamps for video at C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\trimmed-videos\broadcast_2_period_1.mp4


100%|██████████| 600/600 [00:20<00:00, 29.59it/s]


TypeError: post_process_timestamps.<locals>.interpolate_time_array() missing 1 required positional argument: 'frame_rate'

In [3]:
def modify_time_array(time_array, frame_rate):
    """
    Interpolate between indentical time values
    only when it is clear that time remaining will 
    decrease in the next one second of frames.
    """

    seen = {}
    seconds_per_frame = 1/frame_rate
    i = 0
    arr_len = len(time_array)

    while i < arr_len:
        interpolate = False
        curr_value = time_array[i]
        peek_index = i + frame_rate - 1
        if not curr_value is None:
            if (peek_index) < arr_len:
                peek_value = time_array[peek_index]
                if not peek_value is None and peek_value < curr_value:
                    interpolate = True
            else:
                interpolate = True
            if interpolate is True:
                if curr_value in seen:
                    seen[curr_value] += 1
                    time_array[i] -= seen[curr_value] * seconds_per_frame 
                else:
                    seen[curr_value] = 1
                    time_array[i] -= seen[curr_value] * seconds_per_frame
        if time_array[i] < 0:
            time_array[i] = 0.0
        i += 1



# Example usage
# [3.0, 3.0, 3.0, 2.0]
# 

time_array = [None, 3.0, 3.0, 3.0, 3.0,
              2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.2, 1.2, 1.2]
frame_rate = 3  # Frames per second
modify_time_array(time_array, frame_rate)
print(time_array)


[None, 3.0, 3.0, 2.6666666666666665, 2.3333333333333335, 2.0, 2.0, 2.0, 1.6666666666666667, 1.3333333333333335, 1.0, 1.0, 1.0, 1.2, 0.8666666666666667, 0.5333333333333333]


In [10]:
int(2.6)

2